In [1]:
%pylab inline
import pandas as pd
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


/home/benqing/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

In [ ]:
import xgboost

In [3]:
train = pd.read_csv('./train.csv',
                    dtype={'TripType':object,
                           'VisitNumber':int,
                           'Weekday':object,
                           'Upc':float,
                           'ScanCount':int,
                           'DepartmentDescription':object
                           })

In [4]:
#train.TripType = train.TripType.astype('category')
#train.VisitNumber = train.VisitNumber.astype('category')
#train.Weekday = train.Weekday.astype('category')

In [5]:
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [31]:
train.tail()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
647049,39,191346,Sunday,32390001778,1,PHARMACY OTC,1118
647050,39,191346,Sunday,7874205336,1,FROZEN FOODS,1752
647051,39,191346,Sunday,4072,1,PRODUCE,4170
647052,8,191347,Sunday,4190007664,1,DAIRY,1512
647053,8,191347,Sunday,3800059655,1,GROCERY DRY GOODS,3600


In [6]:
train.dtypes

TripType                  object
VisitNumber                int64
Weekday                   object
Upc                      float64
ScanCount                  int64
DepartmentDescription     object
FinelineNumber           float64
dtype: object

In [7]:
train.shape

(647054, 7)

In [8]:
train.describe()

,VisitNumber,Upc,ScanCount,FinelineNumber
count,647054.000000,6.429250e+05,647054.000000,642925.000000
mean,96167.640078,3.060698e+10,1.108878,3726.884567
std,55545.485154,9.120134e+10,0.700776,2780.966158
min,5.000000,8.340000e+02,-12.000000,0.000000
25%,49268.000000,3.400001e+09,1.000000,1404.000000
50%,97074.000000,7.050103e+09,1.000000,3352.000000
75%,144316.000000,3.006531e+10,1.000000,5501.000000
max,191347.000000,9.789707e+11,71.000000,9998.000000


In [9]:
train.Weekday.describe()

count     647054
unique         7
top       Sunday
freq      133975
Name: Weekday, dtype: object

In [10]:
departmentDescriptionCounts = train.DepartmentDescription.value_counts()
#departmentDescriptionCounts.plot()

In [11]:
departmentDescriptionCounts.shape

(68,)

In [12]:
finelineNumberCounts = train.FinelineNumber.value_counts()

In [13]:
finelineNumberCounts.shape

(5195,)

In [14]:
train_sub = train.dropna()

In [15]:
train_sub.shape

(642925, 7)

In [32]:
#X = train.drop(['TripType', 'Upc', 'DepartmentDescription', 'FinelineNumber'], axis=1)
#'VisitNumber',
X = train_sub[['DepartmentDescription', 'Weekday', 'Upc', 'ScanCount', 'FinelineNumber']]

In [33]:
X.head()

,DepartmentDescription,Weekday,Upc,ScanCount,FinelineNumber
0,FINANCIAL SERVICES,Friday,68113152929,-1,1000
1,SHOES,Friday,60538815980,1,8931
2,PERSONAL CARE,Friday,7410811099,1,4504
3,PAINT AND ACCESSORIES,Friday,2238403510,2,3565
4,PAINT AND ACCESSORIES,Friday,2006613744,2,1017


In [34]:
X.dtypes

DepartmentDescription     object
Weekday                   object
Upc                      float64
ScanCount                  int64
FinelineNumber           float64
dtype: object

In [35]:
X_dummies = pd.get_dummies(X)

In [36]:
X_dummies.head()

,Upc,ScanCount,FinelineNumber,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,DepartmentDescription_BAKERY,DepartmentDescription_BATH AND SHOWER,DepartmentDescription_BEAUTY,DepartmentDescription_BEDDING,...,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
0,68113152929,-1,1000,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,60538815980,1,8931,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,7410811099,1,4504,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,2238403510,2,3565,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,2006613744,2,1017,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [37]:
y = train_sub.TripType

In [38]:
y.head()

0    999
1     30
2     30
3     26
4     26
Name: TripType, dtype: object

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, test_size=0.2, random_state=42)

In [40]:
rf_classifier = RandomForestClassifier(n_estimators=75, oob_score=True)

In [41]:
X_train.head()

,Upc,ScanCount,FinelineNumber,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,DepartmentDescription_BAKERY,DepartmentDescription_BATH AND SHOWER,DepartmentDescription_BEAUTY,DepartmentDescription_BEDDING,...,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
306414,2500001922,1,1407,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
643864,5020000880,1,1407,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
79884,83889103073,2,3721,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
126658,3338365020,1,7501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
70975,2700049014,1,3740,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [42]:
rf_classifier = rf_classifier.fit(X_train, y_train)

In [43]:
y_hat_prob = rf_classifier.predict_proba(X_test)

In [44]:
y_hat_prob

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.06293018, ...,  0.03540036,
         0.03949475,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.11437022]])

In [45]:
y_test.head()

141689    44
352749    40
324919    37
456669    39
87890     44
Name: TripType, dtype: object

In [46]:
log_loss(y_test, y_hat_prob)

8.9956648442956038